In [2]:
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
import json
import os

df = pd.read_csv(r"beverly_hills_circuits.csv")

# Create a directory for output GeoJSON files if it doesn't exist
os.makedirs("geojson", exist_ok=True)

# List to hold all GeoDataFrames
gdfs = []

# Function to create GeoDataFrames with all attributes and geometries
def create_geo_dataframe(row):
    # Extract geometries
    def convert_geometry(geometry):
        if isinstance(geometry, str):
            return shape(json.loads(geometry))
        return geometry

    aggregate_geometry = convert_geometry(row['aggregate_geometry'])
    buffer_geometry = convert_geometry(row['buffer_geometry'])
    gesw_segment_geometry = convert_geometry(row['gesw_segment_geometry'])
    
    # Create GeoDataFrames for each geometry type, including all attributes in the row
    gdf_aggregate = gpd.GeoDataFrame([row], geometry=[aggregate_geometry], index=[row['igpe_section_name']])
    gdf_buffer = gpd.GeoDataFrame([row], geometry=[buffer_geometry], index=[row['igpe_section_name']])
    gdf_gesw = gpd.GeoDataFrame([row], geometry=[gesw_segment_geometry], index=[row['igpe_section_name']])
    
    # Append to the list of GeoDataFrames
    gdfs.extend([gdf_aggregate, gdf_buffer, gdf_gesw])

# Apply the function to each row
df.apply(create_geo_dataframe, axis=1)

# Concatenate all the GeoDataFrames into one
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Save the merged GeoDataFrame to a single GeoJSON file
output_file = r"merged_output.geojson"
merged_gdf.to_file(output_file, driver='GeoJSON')

print(f"Merged GeoJSON saved to: {output_file}")


C:\Users\mcveydb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


Merged GeoJSON saved to: geojson/merged_output.geojson
